In [28]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
from firebase_admin import credentials



In [24]:
# Path to your service account JSON file
service_account_key = r"C:\Users\datta\Downloads\fir-project-nlp-firebase.json"

# Initialize the Firebase Admin SDK with explicit credentials
cred = credentials.Certificate(service_account_key)
firebase_admin.initialize_app(cred,name='research_project')


In [33]:
#storing records from csv file into firestore database.

db = firestore.client()
df = pd.read_excel(r'C:\Users\datta\OneDrive\Desktop\RP\search_results_analysis.xlsx')
records = df.to_dict(orient='records')

for record in records:
    db.collection('research_project').add(record)

In [35]:
# Get the collection reference
collection_ref = db.collection('research_project')

# Fetch all documents from the collection
docs = collection_ref.get()



In [38]:
data = []
for doc in docs:
    doc_data = doc.to_dict()
    data.append(doc_data)

In [50]:
df = pd.DataFrame(data)
df=df[['mandate', 'get_vaccine', 'text', 'trust', 'politics', 'ID',
       'family_health', 'child','health_information',
       'personal_decision', 'media', 'side_effect']]
df.head()

,mandate,get_vaccine,text,trust,politics,ID,family_health,child,health_information,personal_decision,media,side_effect
0,0.301636,0.627582,Eve vaccine will save a life,0.520855,0.595995,784.0,0.552873,0.714465,0.479907,0.469474,0.442951,0.438284
1,0.328094,0.264749,I have concerns for those who have recieved,0.473714,0.421723,724.0,0.386119,0.371730,0.483317,0.539276,0.506097,0.378924
2,0.494622,0.892272,"Just keep things clean and organized, when I h...",0.504696,0.524905,359.0,0.567844,0.592648,0.605526,0.570901,0.495645,0.594874
3,0.283007,0.366525,Ima healthy person,0.441875,0.407094,823.0,0.551183,0.658026,0.517945,0.476205,0.390399,0.434088
4,0.412924,0.720978,They believe it's there choice to get the vacc...,0.528991,0.519766,637.0,0.466773,0.569338,0.540406,0.574172,0.502146,0.539771


In [58]:
from firebase_admin import credentials, storage

# Get a reference to the Firebase Storage bucket
bucket = storage.bucket("fir-project-nlp.appspot.com")
# Export DataFrame to CSV
csv_data = df.to_csv(index=False)

# Define the destination filename in the Storage bucket
destination_blob_name = 'research_project/output.csv'

# Create a blob object with the desired filename
blob = bucket.blob(destination_blob_name)

# Upload the CSV data to the Storage bucket
blob.upload_from_string(csv_data, content_type='text/csv')

In [64]:
import datetime
# Get a reference to the Firebase Storage bucket
bucket = storage.bucket("fir-project-nlp.appspot.com")
expiration_time = datetime.datetime.utcnow() + datetime.timedelta(hours=1000)

# Specify the path to your CSV file in Firebase Storage
file_path = "research_project/output.csv"

# Generate a download URL with a temporary token
blob = bucket.blob(file_path)
download_url = blob.generate_signed_url(expiration=expiration_time)  # URL expires in 3 hour

# Print the download URL
print(download_url)


https://storage.googleapis.com/fir-project-nlp.appspot.com/research_project/output.csv?Expires=1689161493&GoogleAccessId=firebase-adminsdk-m34wi%40fir-project-nlp.iam.gserviceaccount.com&Signature=yGTew1L25bAxxtALxzGXeLOJjk48JrnfxR4r5pqdRn44OUS08hcnY1cGkQOFNDXsiGOgXmTFPmzUE9aw2xrwj8i4m5IQn%2Brqjro3nULwX%2BqiqosvcPSjtNSuQTwBrKHZc%2BWMx1%2Bl2lsYctz522%2BEJ8G8lCkHfU06TBVy00rPdoN3wcsdAt2pLpA4Fm4c1bk1CAtrR2lskQdNddvDTsUys6EV4L73jXTDY0gluoiUhMk5N0n%2FSwvgXnQnMtdCcF2sF1UI8tGRALhdE%2BbbCtpdMlwfm6L4gs7mgky66B6Lus9qmvW6mTpIIr%2FGTjKeeSzqpPfUfslIggOZmcELjNfFRQ%3D%3D
